In [7]:
import sys
import os

# Add the parent directory of the current script to the Python path
cwd = os.getcwd()
dirname = os.path.dirname(cwd)
print(cwd)
print(dirname)
sys.path.append(dirname)

print(sys.path)

/Users/idekeradmin/Dropbox/GitHub/agent_evaluation/notebooks
/Users/idekeradmin/Dropbox/GitHub/agent_evaluation
['/Users/idekeradmin/Dropbox/GitHub/agent_evaluation/notebooks', '/opt/anaconda3/lib/python311.zip', '/opt/anaconda3/lib/python3.11', '/opt/anaconda3/lib/python3.11/lib-dynload', '', '/Users/idekeradmin/.local/lib/python3.11/site-packages', '/opt/anaconda3/lib/python3.11/site-packages', '/opt/anaconda3/lib/python3.11/site-packages/aeosa', '/Users/idekeradmin/Dropbox/GitHub/agent_evaluation', '/Users/idekeradmin/Dropbox/GitHub/agent_evaluation']


### Set up the database connection to the test database

In [8]:
from models.analysis_plan import AnalysisPlan
from services.analysisrunner import AnalysisRunner
from app.sqlite_database import SqliteDatabase
from app.config import load_database_config

# Load the db connection details for our test database
# db_type, uri, user, password = load_database_config(path='~/ae_config/test_config.ini')
# self.db = Database(uri, db_type, user, password)

_, database_uri, _, _ = load_database_config()
db = SqliteDatabase(database_uri)

### Load Data and make a dataset

In [9]:
# Load data
import pandas as pd
# This will work even if __file__ is not defined
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print("Project root directory:", project_root)
data_file_path = os.path.join(project_root, "files", "dengue_with_uniprot_top_49.csv")
data = pd.read_csv(data_file_path).to_csv(index=False)

experiment_description = "Dengue virus infection in human cells"

# Create dataset
from models.dataset import Dataset
dengue_dataset = Dataset.create(db, 
                                "dengue data",
                                data, 
                                experiment_description)

# Load the dataset as a new Dataset object
also_dengue_dataset = Dataset.load(db, dengue_dataset.object_id)

# Make a DataFrame from the dataset
from io import StringIO
csv_data = StringIO(also_dengue_dataset.data)
# Read the CSV string into a DataFrame
df = pd.read_csv(csv_data)
df

Project root directory: /Users/idekeradmin/Dropbox/GitHub/agent_evaluation


,GeneID,log2FC gene expression at 24 hours post infection,log2FC gene expression at 48 hours post infection,"siRNA_Screen_Average_Zscore, higher is greater negative effect on dengue infection",log2FC protein abundance at 24 hours post infection,GeneSymbol_48hpi,log2FC protein abundance at 48 hours post infection,dengue proteins that bind this protein
0,3627,NaN,NaN,NaN,11.514163,CXCL10,12.622518,NaN
1,6373,NaN,NaN,NaN,7.872892,CXCL11,11.949547,NaN
2,6355,NaN,NaN,NaN,9.317999,CCL8,11.844292,NaN
3,79931,NaN,NaN,NaN,5.081562,TNIP3,10.909774,NaN
4,3434,3.970117,6.283330,NaN,8.548468,IFIT1,10.798135,NaN
5,202151,NaN,NaN,1.259172,7.063674,RANBP3L,9.733419,NaN
6,282618,NaN,NaN,NaN,7.305892,IFNL1,9.670035,NaN
7,109245082,NaN,NaN,NaN,4.106740,PRAL,9.601910,NaN
8,8519,NaN,NaN,NaN,5.793175,IFITM1,9.348815,NaN
9,135398,NaN,NaN,NaN,6.609377,C6orf141,9.294847,NaN


### Create an LLM and an Analyst

In [ ]:
from models.llm import LLM
from models.analyst import Analyst

context = "Host-virus infection mechanisms, immune response."
prompt_template = """
Please analyze these results of these experiments: {experiment_description}

Here is the data in CSV format:
{data}

Generate a hypothesis providing a mechanistic explanation for some aspect of the data.
Use your knowledge of host-virus infection mechanisms and the immune response to 
produce a NOVEL hypothesis, not one where the relationships and causal effect is already 
known or would be unsurprising to a researcher.
Focus on causal relationships between proteins and their role in the immune response.
Followed the hypothesis with a proposal for a validation experiment.
Choose a name for the hypothesis and put it before the main hypothesis text.
"""

# Create an LLM
# the object should be treated as read-only. The database should be the source of truth.
bob_the_llama3_70b = LLM.create(db, type="Groq", model_name="llama3-70b-8192") 

# Create analyst
# the object should be treated as read-only. The database should be the source of truth.
jane_the_analyst = Analyst.create(db, bob_the_llama3_70b.object_id, context , prompt_template)

In [10]:
jane_the_analyst.object_id

NameError: name 'jane_the_analyst' is not defined